With BERT (DistilBERT)


In [1]:
!pip install matplotlib

In [2]:
!pip install tensorflow

In [3]:
!pip install pdfplumber
!pip install PyPDF2
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.4
    Uninstalling spacy-3.8.4:
      Successfully uninstalled spacy-3.8.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to

In [4]:
import os
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import spacy

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

In [5]:
# Initialize Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained("manishiitg/distilbert-resume-parts-classify")
model = TFAutoModel.from_pretrained("manishiitg/distilbert-resume-parts-classify", from_pt=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [6]:
!unzip /content/drive/MyDrive/HireAIML.zip -d '/content/HireAIML'

Archive:  /content/drive/MyDrive/HireAIML.zip
  inflating: /content/HireAIML/HireAMLE/README.md  
  inflating: /content/HireAIML/HireAMLE/dataset/Job description.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/sample_submission.csv  
  inflating: /content/HireAIML/HireAMLE/dataset/test.csv  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_004.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_005.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_014.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_016.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_017.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_018.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_019.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/candidate_028.pdf  
  inflating: /content/HireAIML/HireAMLE/dataset/testResumes/c

In [7]:
# 1. Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text


In [8]:
# 2. Preprocess Text
def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# 3. Tokenize and Get Embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embedding

# 4. Compute Similarity Scores
def compute_similarity(job_desc_embedding, resume_embeddings):
    similarities = cosine_similarity(job_desc_embedding, resume_embeddings)
    return similarities[0]

In [9]:
# 5. Shortlist Top Candidates
def shortlist_resumes(job_desc, resumes_folder, top_k=5):
    # Step 1: Extract and preprocess job description
    job_desc_text = extract_text_from_pdf(job_desc)
    job_desc_processed = preprocess_text(job_desc_text)
    # print(len(job_desc_processed))
    job_desc_embedding = get_embeddings(job_desc_processed)


    # Step 2: Process and embed resumes
    resume_scores = []
    resume_embeddings = []
    resume_files = os.listdir(resumes_folder)

    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        resume_text = extract_text_from_pdf(resume_path)
        resume_processed = preprocess_text(resume_text)
        resume_embedding = get_embeddings(resume_processed)
        print(resume_embedding.shape)

        # Compute similarity
        score = cosine_similarity(job_desc_embedding, resume_embedding)[0][0]
        resume_scores.append((resume_file, score))
        resume_embeddings.append(resume_embedding)

    # Step 3: Rank resumes by similarity
    resume_scores.sort(key=lambda x: x[1], reverse=True) #reverseTrue is descending order
    print(resume_scores)
    top_candidates = resume_scores[:top_k]

    return top_candidates

# Example Usage
job_description_pdf = "/content/HireAIML/HireAMLE/dataset/Job description.pdf"
resumes_folder = "/content/HireAIML/HireAMLE/dataset/testResumes"

top_candidates = shortlist_resumes(job_description_pdf, resumes_folder)
for candidate in top_candidates:
    print(f"Resume: {candidate[0]}, Similarity Score: {candidate[1]:.2f}")

(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
(1, 768)
[('candidate_039.pdf', np.float32(0.6565534)), ('candidate_103.pdf', np.float32(0.62769735)), ('candidate_059.pdf', np.float32(0.581015)), ('candidate_084.pdf', np.float32(0.5300758)), ('candidate_042.pdf', np.float32(0.52057374)), ('candidate_085.pdf', np.float32(0.49749762)), ('candidate_107.pdf', np.float32(0.4902459)), ('candidate_005.pdf', np.float32(0.48445415)), ('candidate_120.pdf', np.float32(0.48270386)), ('candidate_018.pdf', np.float32(0.471857

In [25]:
print(top_candidates[:5])  # Check the first 5 entries

[('candidate_039.pdf', np.float32(0.6565534)), ('candidate_103.pdf', np.float32(0.62769735)), ('candidate_059.pdf', np.float32(0.581015)), ('candidate_084.pdf', np.float32(0.5300758)), ('candidate_042.pdf', np.float32(0.52057374))]


In [26]:
# import umap
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Extract embeddings
# resume_embeddings = np.array([candidate[2] for candidate in top_candidates])  # Assuming candidate[2] is embedding
# job_desc_embedding = job_desc_embedding.reshape(1, -1)

# # Reduce to 2D for visualization
# reducer = umap.UMAP(n_components=2, random_state=42)
# reduced_embeddings = reducer.fit_transform(np.vstack([job_desc_embedding] + resume_embeddings))

# # Split points for plotting
# all_resumes = reduced_embeddings[1:]  # All resumes
# top_5_indices = [top_candidates.index(candidate) + 1 for candidate in top_candidates[:5]]  # Indices in reduced embeddings
# top_5_points = reduced_embeddings[top_5_indices]  # Top 5 closest resumes

# # Create scatter plot
# plt.figure(figsize=(12, 8))
# sns.scatterplot(x=all_resumes[:, 0], y=all_resumes[:, 1], color="gray", alpha=0.6, label="All Resumes")
# sns.scatterplot(x=[reduced_embeddings[0, 0]], y=[reduced_embeddings[0, 1]], color="red", marker="*", s=300, label="Job Description")
# sns.scatterplot(x=top_5_points[:, 0], y=top_5_points[:, 1], color="blue", s=150, label="Top 5 Closest Resumes", edgecolor="black", linewidth=1.2)

# # Annotate top 5 resumes
# for i, (x, y) in enumerate(top_5_points):
#     plt.text(x, y, f"Top {i+1}", fontsize=12, fontweight='bold', ha='right', color='
